In [ ]:
from IPython.core.display import HTML

with open('style.html', 'r') as file:
    css = file.read()
HTML(css)

In [ ]:
%run 11_imports.ipynb
%run 12_integer_management.ipynb
%run 13_mirroring.ipynb
%run 14_functions.ipynb

# Auswerten eines Schach-Endspiels

Neben der Berechnung der Endspiel-Situationen gilt es zu überprüfen, ob es sich bei den Zügen der KI auch um optimale Halbzüge handelt.
Hierfür werden im Rahmen von diesem Notebook Testszenarien geschrieben, welche die Entscheidungen der KI bewerten.
Zum Vergleich wurde die Stockfish-Engine herangezogen. Stockfish ist die momentan (Stand: 2022) beste Schach-Engine, die frei zur Verfügung steht (Quelle: [chess.com](https://www.chess.com/terms/chess-engine) o. D.).

Insgesamt können vier unterschiedliche Szenarien in diesem Notebook getestet werden. Die Struktur der Tests ist wie folgt aufgebaut:
* ``compare_fen_stockfish``: Dieser Test überprüft für eine mitgegebene FEN jeweils, wie viele Züge die Endspieltabellen und Stockfish zum Gewinnen brauchen.
* ``test_random_boards``: Dieser Test überprüft für eine gegebene Anzahl $n$ zufälliger Situationen, ob Stockfish bessere Ergebnisse liefert.
* ``compare_sequence_stockfish``: Dieser Test überprüft für ein mitgegebenes $n$ alle Situationen, die sich in der Menge $S_n$ befinden, ob Stockfish oder die KI bessere Ergebnisse erzielt.
* ``compare_all_sequences``: Dieser Test überprüft für jegliche Situationen, die einer $S_n$ Menge zugeordnet wurden, ob Stockfish eine Lösung in weniger Zügen bestimmen kann.

Die Ergebnisse der letzten drei Tests werden im Ordner `\Tests` gespeichert.

Zur Durchführung der beschriebenen Tests werden zunächst globale Variablen definiert. Diese lauten wir folgt:
* `g_stockfish_path`: Ein String, der den Pfad zur bereits heruntergeladenen Stockfish-Engine bereitstellt. Sofern dies noch nicht geschehen ist, kann das unter diesem [Link](https://stockfishchess.org/) geschehen.
* `g_verbose`: Ein Boolean, das die zusätzlichen Konsolenausgaben zum Debuggen anzeigt.
* `g_stockfish`: Ein Objekt, das für die Verwendung der Stockfish-Engine benötigt wird.

In [ ]:
g_stockfish_path = "./stockfish/stockfish.exe"
g_verbose = False

g_stockfish = stockfish.Stockfish(g_stockfish_path)

Weiter werden drei globale Variablen für die bereits berechneten $S_n$ Mengen definiert.
Diese Mengen sind in Form einer Datei innerhalb des `S_n_Results`-Ordner zu finden.
Sie werden mit der Funktion `load_data(filename)` aus dem Notebook `14_functions.ipynb` ausgelesen.
Eine detaillierte Beschreibung der Funktion ist dort zu finden.
Bei diesen Variablen handelt es sich um:
* `g_s_n_file`: Ein String, der den Dateinamen ohne die `.chessAI` Endung beinhaltet.
* `g_piece_list`: Eine Liste von `chess.Piece`, die bei der Berechnung der $S_n$ Mengen verwendet wurde.
* `g_s_n_integers`: Eine Liste von Mengen. Diese Mengen enthalten die Integer-Repräsentationen der einzigartigen Spielsituationen der $S_n$ Menge.

In [ ]:
g_s_n_file = "s_n_queen_24_05"

g_piece_list, g_s_n_integers = load_data(g_s_n_file)
g_s_n_integers = gen_all_integers(g_s_n_integers)

## Vorbereitung der Tests

Der Vergleich findet zwischen der KI und der Stockfish-Engine statt.
Hierzu müssen einerseits Funktionen definiert werden, die ein Spiel Stockfish (Schwarz) vs. KI (Weiß) simulieren.
Andererseits gilt es auch Funktionen zu definieren, die ein Spiel Stockfish (Schwarz) vs. Stockfish (Weiß) simulieren.

Mit der Funktion `find_next_move(fen, s_index)` kann nun für eine Partie Stockfish (Schwarz) vs. KI (Weiß) der nächste Halbzug bestimmt werden.
Hierzu werden zunächst zwei Hilfsfunktionen (`ki_move(cur_board, s_index)`, `stockfish_move(cur_board, s_index)`) definiert.

Die Funktion `ki_move(cur_board, s_index)` bestimmt für eine vorgegebene Spielsituation einen Spielzug für die KI.
Dieser Halbzug wird mithilfe der bereits berechneten $S_n$-Mengen bestimmt. Dabei wird in der Spielsituation für jeden möglichen legalen Halbzug geschaut, ob der Folgezug in einer geringeren $S_n$-Menge landet. Die Funktion benötigt hierzu die Parameter:
* `cur_board`: Ein `chess.Board`-Objekt, bei dem gerade Weiß am Zug ist.
* `s_index`: Ein Integer, der das $n$ der $S_n$-Menge widerspiegelt, in dem sich `cur_board` befindet.

Als Rückgabe werden zwei Werte zurückgegeben.
Der erste Wert ist entweder der übergebene `s_index`-1 oder der Wert `-1`.
Handelt es sich um `s_index`-1, wurde ein passender Halbzug für die KI gefunden.
In diesem Fall wird neben dem neuen Index noch der `chess.Move` zurückgegeben.
Sofern der erste Wert `-1` beträgt, konnte kein passender `chess.Move` für die KI gefunden werden und es wird `None` zusätzlich zurückgegeben.

In [ ]:
def ki_move(cur_board, s_index):
    for move in cur_board.legal_moves:
        cur_board.push(move)
        cur_int = to_integer(cur_board, g_piece_list)
        # check if new board is in sequence s_index - 1
        if find_situation_in_sequence(cur_int, [g_s_n_integers[s_index - 1]]) != -1:
            cur_board.pop()
            return s_index - 1, move
        cur_board.pop()
    return -1, None

Das Gegenstück zu `ki_move(cur_board, s_index)` ist die Funktion `stockfish_move(cur_board, s_index)`.
Diese bestimmt für Stockfish den nächsten Halbzug. Hierzu braucht sie ebenfalls zwei Parameter:
* `cur_board`: Ein `chess.Board`-Objekt, bei dem gerade Schwarz am Zug ist.
* `s_index`: Ein Integer, der das $n$ der $S_n$-Menge widerspiegelt, in dem sich `cur_board` befindet.

Die Rückgabe baut sich gleichermaßen aus dem neuen `s_index` und dem ausgeführten `chess.Move` auf. Dabei kann der neue `s_index` auch < `s_index`-1 sein.

In [ ]:
def stockfish_move(cur_board, s_index):
    move = chess.Move.from_uci(g_stockfish.get_best_move())
    cur_board.push(move)
    cur_int = to_integer(cur_board, g_piece_list)
    s_index =  find_situation_in_sequence(cur_int, g_s_n_integers[:s_index])
    cur_board.pop()
    return s_index, move

`find_next_move(fen, s_index)` kombiniert die Funktionen `ki_move(cur_board, s_index)` und `stockfish_move(cur_board, s_index)`.
Sie findet anhand einer übergebenen Spielsituation den nächsten Halbzug, der für den Test Stockfish vs. KI durchgeführt wird.
Dieser kann von Stockfish oder auch von der KI durchgeführt werden.
Hierzu werden folgende Parameter benötigt:
* `cur_board`: Ein `chess.Board`-Objekt, für das der nächste Halbzug gefunden werden soll.
* `s_index`: Ein Integer, der das $n$ der $S_n$-Menge repräsentiert, in der `cur_board` gefunden werden kann.

Die Rückgabe baut sich wie bei `ki_move(cur_board, s_index)` und `stockfish_move(cur_board, s_index)` aus dem `s_index` und dem `chess.Move` auf.

In [ ]:
def find_next_move(cur_board, s_index):
    global g_stockfish

    if g_verbose:
        print("---"+ get_color(cur_board.turn) +":---")
        print("Starting in S" + str(s_index))

    if cur_board.turn:
        s_index, move = ki_move(cur_board, s_index)
    else:
        fen = cur_board.fen()
        g_stockfish.set_fen_position(fen)
        s_index, move = stockfish_move(cur_board, s_index)

    if g_verbose and move is not None:
        print("    Move: " + str(move))
        print("    S" + str(s_index))
        print("Ended in S" + str(s_index))

    return s_index, move

Für das Spielen einer ganzen Spielsituation Stockfish vs. KI wurde die Funktion `all_moves_stock_ai(board)` definiert.
Hierzu wird als Parameter benötigt:
* `board`: Ein `chess.Board`-Objekt, für das die Partie durchgespielt werden soll.

Der Rückgabewert besteht aus einer Liste aus Strings (`moves`).
Diese Strings beschreiben in Universal Chess Interface (UCI) Notation die Züge, die bis zum Beenden der Situation durchgeführt worden sind.

In [ ]:
def all_moves_stock_ai(board):
    moves = []
    cur_int = to_integer(board, g_piece_list)
    s_index = find_situation_in_sequence(cur_int, g_s_n_integers)

    while s_index > 0:
        s_index, next_move = find_next_move(board.copy(), s_index)
        board.push(next_move)
        moves.append(next_move.uci())

    if s_index == -1:
        return None

    return moves

Als Vergleichswert wird die gleiche Spielsituation herangezogen, jedoch werden beide Farben von der Stockfish-Engine gesteuert.
Diese Partie wird mit der Funktion `stockfish_move_list(board)` simuliert.
Sie erhält als Parameter:
* `board`: Ein `chess.Board`-Objekt, das die Ausgangsspielsituation darstellt.

Als Rückgabewert hat diese Funktion ebenfalls eine Liste von Strings (`moves`).
Diese beschreiben die durchgeführten Züge in UCI-Notation.

In [ ]:
def stockfish_move_list(board):
    moves = []

    while not board.is_game_over():
        g_stockfish.set_fen_position(board.fen())
        next_move = chess.Move.from_uci(g_stockfish.get_best_move())
        board.push(next_move)
        moves.append(next_move.uci())

    return moves

# Vergleich Stockfish vs KI

Für den Vergleich der beiden Spielsimulationen werden in den nachkommenden Abschnitten Funktionen definiert, die diese statistisch auswerten.
Dabei wird betrachtet in welchen der Fälle die KI die Partie in weniger Spielzügen als Stockfish beenden konnte.
Gleichermaßen werden auch die Fälle betrachtet, in denen Stockfish besser abgeschlossen hat.

## Statistiken durch den Vergleich

Die Funktion `compare_move_lists(move_count_list)` vergleicht die Anzahl der Halbzüge, die sowohl Stockfish als auch die KI zum Beenden der Spielsituation benötigt haben. Hierfür erhält die Funktion den Parameter:
* `differences`: Diese Liste stellt die Differenzen zwischen der Anzahl an Zügen von Stockfish un der KI dar.

Die Rückgabe der Funktion ist ein Tripel von Integern. Dieses baut sich folgendermaßen auf:
* `equal`: Die Anzahl an Spielen, in denen Stockfish und die KI gleich viele Züge benötigen.
* `ki_better`: Die Anzahl an Spielen, in denen die KI weniger Züge benötigt.
* `stockfish_better`: Die Anzahl an Spielen, in denen Stockfish weniger Züge benötigt.

In [ ]:
def compare_move_lists(differences):
    equal = 0
    ai_better = 0
    stockfish_better = 0
    for  diff in differences:
        if diff == 0:
            equal += 1
        elif diff < 0:
            ai_better += 1
        else:
            stockfish_better += 1
    return equal, ai_better, stockfish_better

Für eine prozentuale Beschreibung der Vergleiche wird die Funktion `get_average_difference(move_count_list)` verwendet.
Diese erhält als Parameter eine List von Tripeln. Die Tripel haben die Form:
* `triple[0]`: Ein Integer, der die Anzahl an Halbzügen darstellt, die die KI zum Beenden einer Situation benötigt hat.
* `triple[1]`: Ein Integer, der die Anzahl an Halbzügen darstellt, die Stockfish zum Beenden einer Situation benötigt hat.
* `triple[2]`: Ein Integer, der die Differenz aus `triple[0]` und `triple[1]` ist.

Die Funktion gibt zwei Float-Werte zurück: `avg_ai_better` und `avg_stock_better`.
Der Float `avg_ai_better` beschreibt den Durchschnittswert um wie viel % der Züge der KI besser waren als die Züge von Stockfish.
Gleichermaßen beschreibt `avg_stock_better` den Durchschnittswert aus Sicht von Stockfish.

In [ ]:
def get_average_difference(move_count_list):
    percentual_ai = []
    percentual_stock = []
    avg_ai_better = 0
    avg_stock_better = 0
    for ai_move, stock_move, diff in move_count_list:
        if diff < 0:
            percentual_ai.append(round(1 - (ai_move / stock_move), 4))
        elif diff > 0:
            percentual_stock.append(round(1 - (stock_move / ai_move), 4))
    if len(percentual_ai) != 0:
        avg_ai_better = sum(percentual_ai) / len(percentual_ai)
    if len(percentual_stock) != 0:
        avg_stock_better = sum(percentual_stock) / len(percentual_stock)
    return avg_ai_better, avg_stock_better

Die zuvor definierten Funktionen dienen der Auswertung der Vergleiche zwischen Stockfish und der KI.
Diese Werte gilt es auch dem Nutzer zur Verfügung zu stellen.
Er kann in Form einer Datei die Statistiken einsehen.
Hierzu wurde die Funktion `write_result_to_file(filename, sequence_index, move_count_list)` implementiert.
Die Parameter der Funktion sind Folgende:
* `filename`: Ein String, in dem der Name der Datei gespeichert in `\Tests` gefunden werden kann (ohne Dateiendung).
* `sequence_index`: Ein Integer, der das $n$ der $S_n$-Menge darstellt.
* `move_count_list`: Eine Liste von Tripeln der Form, die in `get_average_difference(move_count_list)` beschrieben wurde.

In [ ]:
def write_result_to_file(filename, sequence_index, move_count_list):
    equal, ki_better, stockfish_better = compare_move_lists([tripel[2] for tripel in move_count_list])
    avg_ki_better, avg_stock_better = get_average_difference(move_count_list)
    count = ki_better + stockfish_better + equal
    f = open("Tests/" + filename + ".txt", "a+")
    f.write("S_" + str(sequence_index) + ":\n")
    f.write("Stockfish war zu " + str(round((stockfish_better / count) * 100, 2)) + "% besser.\n")
    f.write("Die KI war zu " + str(round((ki_better / count) * 100, 2)) + "% besser.\n")
    f.write("Stockfish und die KI haben zu " + str(
        round((equal / count) * 100, 2)) + "% die gleichen Ergebnisse erzielt.\n")
    f.write("Sofern die KI besser war, hat sie durchschnittlich " + str(
        round(avg_ki_better * 100, 2)) + "% weniger Züge benötigt.\n")
    f.write("Sofern Stockfish besser war, hat sie durchschnittlich " + str(
        round(avg_stock_better * 100, 2)) + "% weniger Züge benötigt.\n")
    f.close()

## Speichern in Universal Chess Interface-Schreibweise der Spielverläufe Stockfish und KI.

Die Funktion `write_both_move_lists(moves_ai, moves_stockfish)` wird in dem Fall benötigt, wenn Stockfish insgesamt weniger Züge benötigt hat als die selbstgeschriebene KI.
Sie ermöglicht es den Spielverlauf anhand einer Move-Historie miteinander zu vergleichen.
Hierzu werden beide Historien nebeneinander in die gleiche Datei wie die zuvor beschriebenen Statistiken geschrieben.
Die Parameter der Funktion sind Folgende:
* `fen`: Ein String, das die Spielsituation in FEN-Notation beschreibt.
* `filename`: Ein String, der den Namen ohne die Dateiendung widerspiegelt.
* `moves_stockfish`: Eine Liste an Strings. Diese enthalten Züge im UCI (Universal Chess Interface) Format. Die Liste wurde bei der Durchführung eines aufgeführten Tests für das Spiel Stockfish vs. Stockfish erstellt.
* `moves_ai`: Eine Liste an Strings. Diese enthalten Züge im UCI Format. Die Liste wurde bei der Durchführung eines aufgeführten Tests für das Spiel KI vs. Stockfish erstellt.


Da die Funktion nur aufgerufen wird, wenn Stockfish weniger Züge als die KI benötigt, ist kein Abgleich der Listen notwendig.

In [ ]:
def write_both_move_lists(fen, filename, moves_stockfish, moves_ai):
    f = open("Tests/" + filename + ".txt", "a+")
    f.write(fen + "\n")
    len_ai = len(moves_ai)
    len_stock = len(moves_stockfish)
    f.write("AI:" + "\t" + "\t" + "\t" + "\t" + " Stockfish:\n")
    for i in range(0, len_stock, 2):
        if i + 1 < len_ai:
            f.write(str(int((i+2)/2)) + ". " + moves_stockfish[i] + " " + moves_stockfish[i+1] +
                    "\t" + " " + str(int((i+2)/2)) + ". " + moves_ai[i] + " " + moves_ai[i+1] + "\n")
        elif i < len_ai:
            f.write(str(int((i+2)/2)) + ". " + moves_stockfish[i] + " " + moves_stockfish[i+1] +
                    "\t" + " " + str(int((i+2)/2)) + ". " + moves_ai[i] + "     " + "\n" )
        else:
            if i + 1 < len_stock:
                f.write(str(int((i+2)/2))+ ". " + moves_stockfish[i] + " " + moves_stockfish[i+1] + "\n")
            elif i < len_stock:
                f.write(str(int((i+2)/2)) + ". " + moves_stockfish[i] + "\n")
    f.close()

Wie die Funktion `write_both_move_lists(fen, filename, moves_stockfish, moves_ai)` wurde die Funktion `display_both_move_lists(moves_stockfish, moves_ai)` definiert.
Diese bildet ebenfalls beide Move-Historien ab mit dem Unterschied, dass sie dieses Mal in der Konsole zu finden sind.
Hierzu werden folgende Parameter benötigt:
* `moves_stockfish`: Eine Liste an Strings. Diese enthalten Züge im UCI Format. Die Liste wurde bei der Durchführung eines aufgeführten Tests für das Spiel Stockfish vs. Stockfish erstellt.
* `moves_ai`: Eine Liste an Strings. Diese enthalten Züge im UCI Format. Die Liste wurde bei der Durchführung eines aufgeführten Tests für das Spiel KI vs. Stockfish erstellt.

In [ ]:
def display_both_move_lists(moves_stockfish, moves_ai):
    len_ai = len(moves_ai)
    len_stock = len(moves_stockfish)
    for i in range(0, len_stock, 2):
        if i + 1 < len_ai:
            print(str(int((i+2)/2)) + ". " + moves_stockfish[i] + " " + moves_stockfish[i+1] +
                  "\t" + " " + str(int((i+2)/2)) + ". " + moves_ai[i] + " " + moves_ai[i+1] + "\n")
        elif i < len_ai:
            print(str(int((i+2)/2)) + ". " + moves_stockfish[i] + " " + moves_stockfish[i+1] +
                  "\t" + " " + str(int((i+2)/2)) + ". " + moves_ai[i] + "     " + "\n" )
        else:
            if i + 1 < len_stock:
                print(str(int((i+2)/2))+ ". " + moves_stockfish[i] + " " + moves_stockfish[i+1] + "\n")
            elif i < len_stock:
                print(str(int((i+2)/2)) + ". " + moves_stockfish[i] + "\n")

# Implementierung der Tests für die $S_n$-Mengen

In diesem Abschnitt werden die zuvor erwähnten Testszenarien implementiert.
Die Tests basieren auf der globalen Variablen `g_s_n_integers`, da aus ihr die Spielsituationen gewählt werden.
Zu Beginn werden jedoch noch eine eigene Exception, aber auch eine Hilfsfunktion definiert.

Bei einem erfolglosen Test wird eine `FailedTest` exception geworfen.
Sie wird nur dann geworfen, wenn innerhalb eines Tests ein TypeError oder ein AttributeError auftritt.
Dieser Fehler bedeutet, dass bei der Erstellung einer move-Liste kein Halbzug für die KI gefunden werden konnte und statt der Liste `None` zurückgegeben wird oder der Spielzug `None` durchgeführt wird.
Durch die `FailedTest` exception gibt es nun die Möglichkeit den fehlgeschlagenen Test noch von anderen Fehlern zu unterscheiden.

In [ ]:
class FailedTest(Exception):
    pass

Bevor die Tests für die $S_n$-Mengen definiert werden, wird zunächst eine Hilfsfunktion implementiert.
Mit der Funktion `ki_vs_stockfish(board, filename)` werden die Berechnungen für Stockfish vs. KI und Stockfish vs. Stockfish angestoßen.
Die generierten Zuglisten werden ausgewertet. Für die Berechnung werden folgende zwei Parameter benötigt:
* `board`: Ein `chess.Board`-Objekt, das die zu testende Spielsituation repräsentiert.
* `filename`: Ein String, der den Dateinamen, in denen die Testergebnisse hinterlegt werden, darstellt.

Als Rückgabewert gibt die Funktion eine Liste von Tripeln zurück. Die Form der Tripel können aus der Beschreibung von der Funktion `get_average_difference(move_count_list)` entnommen werden.

In [ ]:
def ki_vs_stockfish(board, filename):
    ai_moves = all_moves_stock_ai(board.copy())
    stockfish_moves = stockfish_move_list(board.copy())
    move_count = len(ai_moves)
    cmp_move_count = len(stockfish_moves)
    if move_count > cmp_move_count:
        write_both_move_lists(board.fen(), filename, ai_moves, stockfish_moves)
    move_count_t = tuple((move_count, cmp_move_count, move_count - cmp_move_count))
    return move_count_t


Der erste Test sieht den Vergleich der beiden Endspieldatenbanken anhand einer einzelnen Spielsituation vor.
Der Test wird in Form der Funktion `compare_fen_stockfish(fen)` umgesetzt.
Diese erhält als Parameter:
* `fen`: Ein String, der eine Spielsituation in FEN-Notation beschreibt.

Das Ergebnis des Tests wird in Form einer Konsolenausgabe angezeigt.

In [ ]:
def compare_fen_stockfish(fen):
    try:
        board = chess.Board(fen)
        ai_moves = all_moves_stock_ai(board.copy())
        stockfish_moves = stockfish_move_list(board.copy())

        if ai_moves is not None:
            print("AI needed " + str(len(ai_moves)) + " moves to beat Stockfish as Black.")
        else:
            print("AI found no way to beat Black.")

        print("Stockfish needed " + str(len(stockfish_moves)) + " moves to win against itself.")
        if len(stockfish_moves) < len(ai_moves):
            display_both_move_lists(stockfish_moves, ai_moves)
    except (TypeError, AttributeError):
        raise FailedTest("Test failed!")

Eine Erweiterung des ersten Tests findet sich in der Überprüfung mehrerer Spielsituationen wieder.
Daher wurde eine Funktion definiert, die eine vorgegebene Anzahl von zufälligen Spielsituationen abgleicht.
Die Funktion `test_random_boards(count)` erhält dazu einen Parameter:
* `count`: Ein Integer, der für die Anzahl der zu testenden Spielsituation steht.

Das Ergebnis des Tests wird in einer Datei in dem Ordner `\Tests` hinterlegt.
Diese Datei hat einen Namen der Form: `Random_[count]_Compare_[Executiontime].txt`

In [ ]:
import random

def test_random_boards(count):
    try:
        filename = "Random_" + str(count) + "_Compare_" + str(datetime.today().replace(microsecond=0)).replace(":",
                                                                                                               "_") + ".txt"
        count_s_n = len(g_s_n_integers)
        move_count_list = []
        for board_c in range(count):
            rand_s_n = g_s_n_integers[random.randint(0, count_s_n - 2)]
            rand_board_int = random.choice(tuple(rand_s_n))
            rand_board = to_board(rand_board_int, g_piece_list)
            move_count_tuple = ki_vs_stockfish(rand_board, filename)
            move_count_list.append(move_count_tuple)
            clear_output()
            print("Analyzed " + str(board_c + 1) + "/" + str(count))

        write_result_to_file(filename, "random", move_count_list)
    except (TypeError, AttributeError):
        raise FailedTest("Test failed!")

In [ ]:
test_random_boards(100)

Die Funktion `compare_sequence_stockfish(sequence_index, g_filename=None)` stellt die Überprüfung einer ganzen $S_n$-Menge zur Verfügung.
Hierzu wird jede Spielsituation einer $S_n$-Menge einmal mit Stockfish vs. KI und ein zweites Mal mit Stockfish vs. Stockfish simuliert.
Die Funktion erhält für die Durchführung als Parameter:
* `sequence_index`: Ein Integer, der das $n$ der zu überprüfenden $S_n$-Menge darstellt.
* `g_filename` (optional): Ein String mit dem globalen Dateinamen, falls mehrere $S_n$-Mengen überprüft werden sollen und die Ergebnisse in eine Datei geschrieben werden sollen.

Das Ergebnis wird beim Vergleich einer einzelnen $S_n$-Menge in eine Datei in `\Tests` mit dem Dateinamen der Form: `S_[sequence_index]_Compare_[Executiontime].txt` geschrieben.
Sofern ein `g_filename` übergeben wurde, wird das Ergebnis in diese Datei geschrieben.

In [ ]:
def compare_sequence_stockfish(sequence_index, g_filename=None):
    try:
        s_n = g_s_n_integers[sequence_index]
        move_count_list = []
        length = len(s_n)
        r = 0
        if g_filename is None:
            filename = "S_" + str(sequence_index) + "_Compare_" + str(datetime.today().replace(microsecond=0)).replace(":",                                                                                                                   "_") + ".txt"
        else:
            filename = g_filename

        for board_int in s_n:
            clear_output()
            board = to_board(board_int, g_piece_list)
            move_count_tuple = ki_vs_stockfish(board, filename)
            move_count_list.append(move_count_tuple)
            print("Comparing S_" + str(sequence_index) + ":")
            print("Compared " + str(r + 1) + "/" + str(length))
            r += 1
        write_result_to_file(filename, sequence_index, move_count_list)
    except (TypeError, AttributeError):
        raise FailedTest("Test failed!")

In [ ]:
compare_sequence_stockfish(4)

Sofern keine einzelne Überprüfung ausgewählter Spielsituationen überprüft werden soll, bietet die Funktion `compare_all_sequences()` die Option alle berechneten Spielsituationen in den $S_n$-Mengen zu überprüfen.
Hierzu wird die Funktion `compare_sequence_stockfish(sequence_index, g_filename=None)` aufgerufen.

Das Ergebnis wird in eine Datei der Form: `All_S_Compare_[Executiontime].txt` geschrieben.

In [ ]:
def compare_all_sequences():
    try:
        filename = "All_S_Compare_" + str(datetime.today().replace(microsecond=0)).replace(":", "_") + ".txt"
        for i in range(len(g_s_n_integers)):
            print("Comparing S_" + str(i) + "...")
            compare_sequence_stockfish(i, filename)
    except FailedTest:
        raise FailedTest("Test failed!")

In [ ]:
compare_all_sequences()
